# Synthetic Ramesside Star Clocks

In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import math
import star_chart_spherical_projection as scsp
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun, Angle, Longitude
from sunpy.coordinates import frames, sun

# I want get_sun to shut up so ignoring warnings:
import warnings
warnings.simplefilter('ignore', UserWarning)

## Functions

In [7]:
def StarRiseSet(jd, starAlt, deg):
    # create return arrays
    star_rise = np.zeros(int(len(jd)/360), dtype=int)
    star_set = np.zeros(int(len(jd)/360), dtype=int)
    # loop through each day
    for i in range(360, len(jd) + 1, 360):
        # isolate data for one day
        dailyStarAlt = starAlt[i - 360 : i]
        # is the star visible at the beginning of the day
        starVis = dailyStarAlt[0] > deg
        # make array of booleans
        bool_arr = dailyStarAlt > deg   
        # find two indices of change
        ind1 = np.argwhere(bool_arr != starVis)[0][0]
        if len(np.argwhere(bool_arr[ind1:]== starVis)) != 0: # edge case of next index is in next day
            ind2 = np.argwhere(bool_arr[ind1:]== starVis)[0][0] + ind1
        else:
            ind2 = 360    
        # assign indices
        if starVis:
            star_set[int((i-360)/360)] = int(-360 + i  + ind1)
            star_rise[int((i-360)/360)] = int(-360 + i  + ind2)
        else:
            star_set[int((i-360)/360)] = int(-360 + i  + ind2)
            star_rise[int((i-360)/360)] = int(-360 + i  + ind1)
    return (star_rise, star_set)

def SunRiseSet(jd, SunAlt, deg):
    
    '''
    A function to create a list of indices where the Sun rises and sets in a given year. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    NOTE: as written, this code assumes that data is collected every 4 minutes. 
    To change this, change number to number of collection intervals per day! (360 = 24 * 60/4)
    Inputs: 
        jd = Julian date
        SunAlt = the altitude of the Sun
    Outputs:
        sunriseset = indices of sunrise and sunset in the jd & date columns
    '''
    
    sunriseset = []
    for i in range(360, len(jd), 360):
        temp = []
        for j in range(i - 360, i):
            if SunAlt[j] <= deg + 0.4 and SunAlt[j] >= deg - 0.4:
                if len(temp) == 0: 
                    temp.append(j)
                elif temp[-1] != j - 1:
                    temp.append(j)
        sunriseset.append(temp)
    return sunriseset


def isStarVisible(sunSet, sunRise, starAlt):
    vis_arr = np.full(365, True)
    max_alt_arr = np.zeros(364)
    for i in range(0, 364):
        maxalt = max(starAlt[sunSet[i]:sunRise[i + 1]])
        max_alt_arr[i] = maxalt
        #print(maxalt)
        if maxalt < 0:
            vis_arr[i] = False
    return(max_alt_arr, vis_arr)


def MaxMinAltAz(jd, sunriseset, DecAz, DecAlt):
    
    '''
    A function to create lists of minimum and maximum azimuths and altitudes of the decan. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    Inputs: 
        direct = string with the directory where the .txt file is located
        filename = string with name of file (name + month + year)
        jv = Julian date
        sunriseset = indices of sunrize and sunset in the jd & date columns
    Outputs:
        sunriseset = indices of sunrize and sunset in the jd & date columns
        days = list of indices when it's daylight 
        minaz, maxaz = minimum and maximum azimuths of the decan per night
        minalt, maxalt = minimum and maximum altitudes of the decan per night
        riseaz, setaz = azimuth of decan at rise & set
        risealt, setalt = altitude of decan at rise & set
    '''
    maxalt = []
    minalt = []
    maxaz = []
    minaz = []
    riseaz = []
    setaz = []
    risealt = []
    setalt = []
    days = []
    for i in range(0, int(len(jd)/360) - 1):
        sset = sunriseset[i][1]
        srise = sunriseset[i + 1][0]
        maxalt.append(max(DecAlt[sset:srise]))
        minalt.append(min(DecAlt[sset:srise]))
        maxaz.append(max(DecAz[sset:srise]))
        minaz.append(min(DecAz[sset:srise]))
        riseaz.append(DecAz[srise])
        setaz.append(DecAz[sset])
        risealt.append(DecAlt[srise])
        setalt.append(DecAlt[sset])
        days.append(DecAlt[srise:sset])
    return(days, minaz, maxaz, minalt, maxalt, riseaz, setaz, risealt, setalt)
def horizonBins(horizon, bsize, gsize):
    # if gap size is zero, return as usual
    if gsize == 0:
        return((np.linspace(horizon[0], horizon[1], 8), 1))
    # otherwise, divide into smallest common denominator
    num = 7 * bsize + 6 * gsize + 1
    horizon_bins = np.linspace(horizon[0], horizon[1], num)
    #print(len(horizon_bins))
    # select indices to merge by bin/gap size
    ## first gaps
    ind_gap = np.zeros(14)
    ind_gap[1::2] = np.arange(0,7)
    ind_gap[::2] = np.arange(0,7)
    #print(ind_gap)
    ## then bins
    ind_bin = np.zeros(14)
    ind_bin[0:13] = ind_gap[1:]
    ind_bin[-1] = 7 
    #print(ind_bin)
    # add together with bin/gap sizes
    inds = ind_bin * bsize + ind_gap * gsize
    inds = inds.astype(int)
    #print(inds)
    # return only those bin and gap indices
    return((horizon_bins[inds], 2))

# def synRSC1star(date_ind, horizon, bsize, gsize, sunSet, sunRise, starAz, starVis):
#     '''Function which, when given a date, size of horizon, and star data*, 
#     makes a synthetic Ramesside Star Clock.'''
#     # define horizon bin limits (assuming equal binsize)
#     (horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
#     # define time bin limits (assuming equal binsize)
#     sset = sunSet[date_ind]
#     sris = sunRise[date_ind + 1]
#     #time_bins = np.linspace(jd[sset], jd[srise], 14)
#     time_ind_bins = np.round(np.linspace(sset, sris, 13))
#     # bin star as appropriate 
#     rsc_table = []
#     for i in range(0, 13):
#         ind = int(time_ind_bins[i])
#         if starVis[date_ind]:
#             az_list = starAz[ind]
#             row = np.histogram(az_list, horizon_bins)[0][0:-1:skip]
#             #print("hour " + "{:02d}".format(i), row)
#         else:
#             row = np.zeros(7)       
#         rsc_table.append(row)
#     return(np.array(rsc_table))

def isStarInWindow(alt_window, starAlt):
    '''
    Function to check whether star is in a given altitude window.
    '''
    (alt_min, alt_max) = alt_window
    if starAlt >= alt_min and starAlt <=alt_max:
        return(True)
    else:
        return(False)


def synRSC1star(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starName, starAz, starAlt, starVis):
    '''Function which, when given a date, size of horizon, and single star's data*, 
    makes a synthetic Ramesside Star Clock.'''
    # define horizon bin limits (assuming equal binsize)
    (horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
    # define time bin limits (assuming equal binsize)
    sset = sunSet[date_ind]
    sris = sunRise[date_ind + 1]
    #time_bins = np.linspace(jd[sset], jd[srise], 14)
    time_ind_bins = np.round(np.linspace(sset, sris, 13))
    # bin star as appropriate 
    rsc_table = []
    for i in range(0, 13):
        ind = int(time_ind_bins[i]) # is this the index I want to be passing? 
        inWindow = isStarInWindow(alt_window, starAlt[ind])
        if starVis[date_ind] and inWindow:
            az_list = starAz[ind]
            row = np.histogram(az_list, horizon_bins)[0][::skip]
            #print(date_ind)
            #print("in Window=" + str(inWindow)+", az_list = "+ str(az_list))
        else:
            row = np.zeros(7, dtype=int)  
        rsc_table.append(row)
    return(np.array(rsc_table))

def synRSC(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist, starAzlist, starAltlist, starVislist):
    '''
    Function to create synthetic Ramesside Star Clocks given data from several stars.
    '''
    # how many stars
    df = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate over stars
    for i in range(len(starlist)):
        temp_table = synRSC1star(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist[i], starAzlist[i], starAltlist[i], starVislist[i])
        inds = np.argwhere(temp_table == 1)
        for ind in inds:
            df.at[ind[0], ind[1]] += (starlist[i] + " ")   
    df.columns = [-3, -2, -1, 0, 1, 2, 3]
    return df

def mag_data(df, mag_dict):
    '''
    Given a data frame made with synRSC and a name-to-magnitude value dictionary, 
    this function will select the brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock.  
    '''
    # data frame to save magnitude-selected data
    df_mag = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for magnitude
    for i in range(0, 13):
        sname = ""
        min_mag = 10 # all human visible magnitudes should be higher than this 
        # iterate through columns in row ( = horizon bins)
        for j in range(-3, 4):
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            for item in dlist:
                if mag_dict[item] < min_mag:
                    min_mag = mag_dict[item] # update brightest available star
                    cind = j + 3 # column index
                    sname = item # star name 
        if len(sname) > 1:                     
            df_mag.at[i, cind] = sname
    df_mag.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_mag)


def name_or_mag_data(df, mag_dict, known_stars):
    '''
    Given a data frame made with synRSC, a name-to-magnitude value dictionary, and a known-star dictionary, 
    this function will select the known brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock;
    if no known stars, it defaults to brightest. 

    BEWARE: call this the alpha version of this function; it is the FARTHEST thing from elegant or optimized.   
    '''
    # data frame to save magnitude-selected data
    df_magname = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for "known stars", then magnitude
    for i in range(0, 13): # for each row
        row_list = [] #list of stars in row
        for j in range(-3, 4): # iterate through columns in row ( = horizon bins)
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            row_list += dlist
        scand_list = [] # list of known candidate stars 
        for scand in row_list:
            if scand in known_stars:
                scand_list.append(scand)     
        # CASE 1: no previously known stars, choose by magnitude and add to known star list 
        if len(scand_list) == 0: 
            sname=''
            min_mag = 10 # all human visible magnitudes should be higher than this 
            for j in range(-3, 4):
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                for item in dlist:
                    if mag_dict[item] < min_mag:
                        min_mag = mag_dict[item] # update brightest available star
                        cind = j + 3 # column index
                        sname = item # star name 
            if len(sname) > 1: # if it's found *no* stars, leave blank                
                df_magname.at[i, cind] = sname  
                known_stars[sname] = "K" + str(len(known_stars)).zfill(2) # update known star dictionary           
        # CASE 2: one known star, choose that one
        elif len(scand_list) == 1:
            sname = scand_list[0]
            for j in range(-3, 4): # iterate through columns in row ( = horizon bins)
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                if sname in dlist:
                    cind = j + 3 # column index
                    df_magname.at[i, cind] = sname 
                    #known_stars[sname] = "K" + str(len(known_stars)).zfill(2) # update known star dictionary 
        # CASE 3: several known stars, choose the brightest one 
        else:
            # first find the brightest available star
            min_mag = 10 # all human visible magnitudes should be higher than this 
            for scand in scand_list:
                if mag_dict[scand] < min_mag:
                    min_mag = mag_dict[scand] # update brightest available star
                    sname = scand # star name 
                    #known_stars[sname] = "K" + str(len(known_stars)).zfill(2) # update known star dictionary 
            for j in range(-3, 4): # now find position of star
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                if sname in dlist:
                    cind = j + 3 # column index
                    df_magname.at[i, cind] = sname 
    # return                 
    df_magname.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_magname, known_stars)    



## Import Data & Make Necessary Structures

In [3]:
### Set the directory
direct = os.path.abspath(os.path.join(os.getcwd(), os.pardir))  # parent of current working directory
direct = direct + '/StarLists/' # directory where the .txt files go
filename = "RealSky/hippdata1300BC.txt" # which data file (generated by mockrun.py)
#filename = "RandSky/mockdata_200_1300BC-Jan-29-2024_1422.txt" # which data file (generated by mockrun.py)

In [4]:
decanOutput = pd.read_csv(direct + filename, sep = "|")

# get header 
header = decanOutput.keys()

# get list of star names
starlist = []
for star_name in header[4:-1:2]:
    starlist.append(star_name[0:-8])
# get standard data
jd = decanOutput[header[0]].to_numpy()
hrd = decanOutput[header[1]]
sunAz = decanOutput[header[2]].to_numpy()
sunAlt = decanOutput[header[3]].to_numpy()

# get star data
num_decs = int( (len(header) - 4) / 2 ) # how many stars in list 
starsAz = np.zeros((num_decs, len(jd)))
starsAlt = np.zeros((num_decs, len(jd)))
for i in range(0, num_decs):
    starsAz[i, :] = decanOutput[header[4 + 2 * i]].to_numpy()
    starsAlt[i, :] = decanOutput[header[5 + 2 * i]].to_numpy()
   
# get sunrise and sunset times
(sunRise, sunSet) = StarRiseSet(jd, sunAlt, -12)
sunAzSet = sunAz[sunSet]

# get star rise and set times
rise_alt_deg = 10 # define starrise in muber of degrees above horizon 
starAzRiseList = np.zeros((num_decs, len(sunRise)))
starVisList = np.full((num_decs, len(sunRise)), True)
starMaxAltList = np.zeros((num_decs, len(sunRise)-1))
# loop over all decans
for i in range(0, num_decs):
    # create visibility and maximum altitude array
    if np.min(starsAlt[i]) >= rise_alt_deg: 
        # if the star is circumpolar at the given location
        starVisList[i,:] = np.full((365,), True)
        starMaxAltList[i,:] = np.max(starsAlt[i])
    elif np.max(starsAlt[i]) < rise_alt_deg:
        # if a star never rises above rise_alt_deg
        starVisList[i,:] = np.full((365,), False)
        starMaxAltList[i,:] = np.max(starsAlt[i])
    else: 
        # otherwise, check when the star isn't visible
        (starRise, starSet) = StarRiseSet(jd, starsAlt[i], rise_alt_deg)    #???
        starAzRise = starsAz[i, starRise]                                   #???
        starAzRiseList[i, :] = starAzRise                                   #???
        (maxAlt, starVis) = isStarVisible(sunSet, sunRise, starsAlt[i])
        starVisList[i,:] = starVis
        starMaxAltList[i,:] = maxAlt


## Create Synthetic RSCs

In [16]:
alt_window = (0, 45) # degrees 
horizon = (170, 190) # Note: MUST have smaller number first; 
bsize = 2 # bin size (must be 1 if gsize = 0)
gsize = 1 # gap size (relative to binsize) 

# make dictionary of names and magnitudes (if not already in memory) 
# THIS NEEDS TO BE GENERALIZED --> currently inconsistent between real and fake data \
# I should really rethink my folder structure
name_df = pd.read_csv(direct + '/RealSky/ICs/star_data_names.csv', index_col=None, header=0, names=['Name', 'RA', 'Dec', 'Mag'])
mag_dict = name_df.set_index('Name')['Mag'].to_dict()

In [17]:
# Creating Excel Writer Object from Pandas  
folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/SynRSC/" # parent of current working directory" + ./SynRSC/"
wname = 'synRSC_FS_hor' + str(horizon[0]) + '-' + str(horizon[1]) + '_b=' + str(bsize) + '_g=' + str(gsize) + '_' + str(num_decs) + '.xlsx'

writer = pd.ExcelWriter(folder + wname, engine='xlsxwriter')     
workbook=writer.book
worksheet=workbook.add_worksheet('RSCs')
writer.sheets['RSCs'] = worksheet
worksheet2=workbook.add_worksheet('Mag Select')
writer.sheets['Mag Select'] = worksheet2
worksheet3=workbook.add_worksheet('Name Select')
writer.sheets['Name Select'] = worksheet3
# Format test
format = workbook.add_format()
format.set_font_size(11)
# Write down some important data
worksheet.write(0, 0, "horizon is " + str(horizon), format)
worksheet.write(1, 0, "alt window is " + str(alt_window), format)
worksheet.write(2, 0, "bsize = " + str(bsize), format)
worksheet.write(3, 0, "gsize = " + str(gsize), format)

# Write data (saved in /SynRsc folder)
known_stars_dict = {}
for i in range(0, 24):
    date = i * 15 # days from first day in decan data
    df = synRSC(date, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist, starsAz, starsAlt, starVisList)
    #df_test.to_excel('output' + str(date) + '.xlsx', index=False)
    df.to_excel(writer, sheet_name='RSCs',startrow= i * 15 + 5, startcol=0)   
    worksheet.write(i * 15 + 5,  0, "Table " + str(i + 1), format)
    #another_df.to_excel(writer,sheet_name='RSCs',startrow=20, startcol=0) 
    #add mag data
    df_mag = mag_data(df, mag_dict)
    df_mag.to_excel(writer, sheet_name='Mag Select',startrow= i * 15 + 5, startcol=0) 
    worksheet2.write(i * 15 + 5,  0, "Table " + str(i + 1), format)
    # add name or mag data
    (df_name, known_stars_dict) = name_or_mag_data(df, mag_dict, known_stars_dict)
    df_name.to_excel(writer, sheet_name='Name Select',startrow= i * 15 + 5, startcol=0) 
    worksheet3.write(i * 15 + 5,  0, "Table " + str(i + 1), format)
writer.close()    

In [15]:
print(len(known_stars_dict))
known_stars_dict

78


{'H019747': 'K00',
 'H027072': 'K01',
 'H033579': 'K02',
 'H035264': 'K03',
 'H045080': 'K04',
 'H059196': 'K05',
 'H060718': 'K06',
 'H067464': 'K07',
 'H071681': 'K08',
 'H016537': 'K09',
 'H032768': 'K10',
 'H041037': 'K11',
 'H042913': 'K12',
 'H061932': 'K13',
 'H068702': 'K14',
 'H071683': 'K15',
 'H085792': 'K16',
 'H023685': 'K17',
 'H025985': 'K18',
 'H034444': 'K19',
 'H039757': 'K20',
 'H061585': 'K21',
 'H071908': 'K22',
 'H082396': 'K23',
 'H086670': 'K24',
 'H048002': 'K25',
 'H052419': 'K26',
 'H076297': 'K27',
 'H085696': 'K28',
 'H036377': 'K29',
 'H088635': 'K30',
 'H090496': 'K31',
 'H046701': 'K32',
 'H059747': 'K33',
 'H077952': 'K34',
 'H045941': 'K35',
 'H085927': 'K36',
 'H089642': 'K37',
 'H095241': 'K38',
 'H101772': 'K39',
 'H090185': 'K40',
 'H071352': 'K41',
 'H095347': 'K42',
 'H105881': 'K43',
 'H075141': 'K44',
 'H108085': 'K45',
 'H062322': 'K46',
 'H074946': 'K47',
 'H083081': 'K48',
 'H093683': 'K49',
 'H114341': 'K50',
 'H113136': 'K51',
 'H107556': 

# Scratch

### Checking bins and gaps

In [18]:
def horizonBins(horizon, bsize, gsize):
    # if gap size is zero, return as usual
    if gsize == 0:
        return((np.linspace(horizon[0], horizon[1], 8), 1))
    # otherwise, divide into smallest common denominator
    num = 7 * bsize + 7 * gsize
    horizon_bins = np.linspace(horizon[0], horizon[1], num)
    # select indices to merge by bin/gap size
    ## first gaps
    ind_gap = np.zeros(14)
    ind_gap[1::2] = np.arange(0,7)
    ind_gap[::2] = np.arange(0,7)
    ## then bins
    ind_bin = np.zeros(14)
    ind_bin[0:13] = ind_gap[1:]
    ind_bin[-1] = 7
    # add together with bin/gap sizes
    inds = ind_bin * bsize + ind_gap * gsize
    inds = inds.astype(int)
    # return only those bin and gap indices
    return((horizon_bins[inds], 2))


def synRSC1star(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starName, starAz, starAlt, starVis):
    '''Function which, when given a date, size of horizon, and single star's data*, 
    makes a synthetic Ramesside Star Clock.'''
    # define horizon bin limits (assuming equal binsize)
    (horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
    # define time bin limits (assuming equal binsize)
    sset = sunSet[date_ind]
    sris = sunRise[date_ind + 1]
    #time_bins = np.linspace(jd[sset], jd[srise], 14)
    time_ind_bins = np.round(np.linspace(sset, sris, 13))
    print(sset)
    print(sris)
    print(time_ind_bins)
    #print(time_ind_bins)
    # bin star as appropriate 
    rsc_table = []
    for i in range(0, 13):
        ind = int(time_ind_bins[i]) # is this the index I want to be passing? 
        inWindow = isStarInWindow(alt_window, starAlt[ind])
        if starVis[date_ind] and inWindow:
            az_list = starAz[ind]
            row = np.histogram(az_list, horizon_bins)[0][::skip]
            print(np.histogram(az_list, horizon_bins)[0])
            print(row)
            #print(date_ind)
            #print("in Window=" + str(inWindow)+", az_list = "+ str(az_list))
        else:
            row = np.zeros(7, dtype=int)  
        rsc_table.append(row)
    return(np.array(rsc_table))    

In [347]:
alt_window = (0, 45) # degrees 
horizon = (170, 208) # Note: MUST have smaller number first; 
bsize = 1 # bin size (must be 1 if gsize = 0)
gsize = 4 # gap size (relative to binsize) 

dhour = 0.04166666674427688 #in Julian dates

(horizon_bins, skip) = horizonBins(horizon, bsize, gsize)

print(horizon_bins)

len(horizon_bins)

[170.         171.22580645 176.12903226 177.35483871 182.25806452
 183.48387097 188.38709677 189.61290323 194.51612903 195.74193548
 200.64516129 201.87096774 206.77419355 208.        ]


14

In [346]:
208 - 170

38

In [27]:
def horizonBins(horizon, bsize, gsize):
    # if gap size is zero, return as usual
    if gsize == 0:
        return((np.linspace(horizon[0], horizon[1], 8), 1))
    # otherwise, divide into smallest common denominator
    num = 7 * bsize + 7 * gsize #+ 1
    horizon_bins = np.linspace(horizon[0], horizon[1], num)
    #print(len(horizon_bins))
    # select indices to merge by bin/gap size
    ## first gaps
    ind_gap = np.zeros(14)
    ind_gap[1::2] = np.arange(0,7)
    ind_gap[::2] = np.arange(0,7)
    #print(ind_gap)
    ## then bins
    ind_bin = np.zeros(14)
    ind_bin[0:13] = ind_gap[1:]
    ind_bin[-1] = 7 
    #print(ind_bin)
    # add together with bin/gap sizes
    inds = ind_bin * bsize + ind_gap * gsize
    inds = inds.astype(int)
    #print(inds)
    # return only those bin and gap indices
    return((horizon_bins[inds], 2))

alt_window = (0, 45) # degrees 
horizon = (170, 190) # Note: MUST have smaller number first; 
bsize = 1 # bin size (must be 1 if gsize = 0)
gsize = 2 # gap size (relative to binsize) 
dhour = 0.04166666674427688 #in Julian dates
(horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
print(horizon_bins)

[170. 171. 173. 174. 176. 177. 179. 180. 182. 183. 185. 186. 188. 189.]


In [301]:
n1 = horizon_bins[2] - horizon_bins[1]
n2 = horizon_bins[3] - horizon_bins[2]

print(n1)
print(n2)
print(n1/n2)
print(np.round(n1/n2,6)==np.round(gsize/bsize,6))

1.0144927536231876
9.130434782608717
0.11111111111111077
True


In [300]:
np.round(n1/n2,6)

0.111111

In [295]:
1/9

0.1111111111111111

In [20]:
date = 0 # days from first day in decan data

si = starlist.index('H013701') # a star I know will appear with bsize = 1 gsize = 0

synRSC1star(date, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist[si], starsAz[si], starsAlt[si], starVisList[si])

274
450
[274. 289. 303. 318. 333. 347. 362. 377. 391. 406. 421. 435. 450.]
[0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]


array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [259]:
time_inds = [274, 289, 303, 318, 333, 347, 362, 377, 391, 406, 421, 435, 450]

for i in range(1, len(time_inds)):
    print(time_inds[i] - time_inds[i-1])
    print((jd[time_inds[i]] - jd[time_inds[i-1]])/dhour)

#(jd[318] - jd[289]) / dhour

15
1.0
14
0.9333333374311527
15
0.9999999944120646
15
1.0000000055879354
14
0.9333333318432172
15
0.9999999552965165
15
1.0
14
0.9333333318432172
15
1.0
15
1.0
14
0.9333333318432172
15
1.0


In [187]:
horizon_bins.shape

(8,)

### Other

Ha, it works! Some notes on things to do:

- general filestructure cleanup
    - the dictionary importing is hard coded right now--have it depends on direct, folder, filename, etc 
    - speaking of which, let's start naming files and folders similarly, so that IC filename --> decanOpy output folder --> SynRSC folder 
    - Clean up extraneous files and backup on git!
- maybe separate IC generation from decanOpy run?
    - this would involve figuring out how to point to a different star data csv for the real sky, which I have to do anyway
- RSC generator should be wrapped up in at least one more layer of functions
    - functions can also be stored in separate file 
    - in fact, decanO.py and co should probably go in their own folder and be imported properly (ayayay)
    - go through and comment the functions
    - generate readthedocs!
- RSC generator should have more choices
    - magnitude
    - "known star" THEN magnitude 
    - closest to center of bin 


    - Fake stars
        - maximum magnitude 4 
        - make sure not too many bright stars -- maybe not a uniform randomness?


        Largest Hipaprchos number for real data is 116727:
         -  name data as letter + 6 numbers, always a string 7-elements long


In [107]:
# inv_mag_dict = dict((v, k) for k, v in mag_dict.items())  ### invert dictionary

# # Python code to demonstrate 
# # finding duplicate values from a dictionary

# # initialising dictionary
# ini_dict = mag_dict


# # printing initial_dictionary
# print("initial_dictionary", str(ini_dict))

# # finding duplicate values
# # from dictionary
# # using a naive approach
# rev_dict = {}

# for key, value in ini_dict.items():
# 	rev_dict.setdefault(value, set()).add(key)
	
# result = [key for key, values in rev_dict.items()
# 							if len(values) > 1]

# # printing result
# print("duplicate values", str(result))


In [110]:
test_dict = {}

test_dict["name1"] = "name2"

print(test_dict)

{'name1': 'name2'}


In [27]:
testd_df_save = df

In [ ]:
def mag_data(df, mag_dict):
    '''
    Given a data frame made with synRSC and a name-to-magnitude value dictionary, 
    this function will select the brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock.  
    '''
    # data frame to save magnitude-selected data
    df_mag = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for magnitude
    for i in range(0, 13):
        sname = ""
        min_mag = 10 # all human visible magnitudes should be higher than this 
        # iterate through columns in row ( = horizon bins)
        for j in range(-3, 4):
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            for item in dlist:
                if mag_dict[item] < min_mag:
                    min_mag = mag_dict[item] # update brightest available star
                    cind = j + 3 # column index
                    sname = item # star name 
        if len(sname) > 1:                     
            df_mag.at[i, cind] = sname
    df_mag.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_mag)

In [44]:
def mag_or_name_data(df, mag_dict, known_stars):
    '''
    Given a data frame made with synRSC and a name-to-magnitude value dictionary, 
    this function will select the brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock.  
    '''
    # data frame to save magnitude-selected data
    df_magname = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for magnitude
    for i in range(0, 13):
        sname = ""
        min_mag = 10 # all human visible magnitudes should be higher than this 
        # iterate through columns in row ( = horizon bins)
        for j in range(-3, 4):
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            for item in dlist:
                if mag_dict[item] < min_mag:
                    min_mag = mag_dict[item] # update brightest available star
                    cind = j + 3 # column index
                    sname = item # star name             
        # ok somewhere around here, I want to start making a dictionary of "known stars for comparison"
        scandlist = []; # empty list of star candidates
        for scand in dlist:
            if scand in known_stars:
                scandlist.append(scand)
            
        if len(sname) > 1: # if it's found *no* stars, leave blank
            # if no known stars                    
            df_magname.at[i, cind] = sname #choose the magnitude one
            # else
            # choose the known star
    df_magname.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_magname)

In [105]:

def name_or_mag_data(df, mag_dict, known_stars):
    '''
    Given a data frame made with synRSC, a name-to-magnitude value dictionary, and a known-star dictionary, 
    this function will select the known brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock;
    if no known stars, it defaults to brightest. 

    BEWARE: call this the alpha version of this function; it is the FARTHEST thing from elegant or optimized.   
    '''
    # data frame to save magnitude-selected data
    df_magname = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for "known stars", then magnitude
    for i in range(0, 13): # for each row
        row_list = [] #list of stars in row
        for j in range(-3, 4): # iterate through columns in row ( = horizon bins)
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            row_list += dlist
        scand_list = [] # list of known candidate stars 
        for scand in row_list:
            if scand in known_stars:
                scand_list.append(scand)
        # CASE 1: no previously known stars, choose by magnitude
        if len(scand_list) == 0: 
            min_mag = 10 # all human visible magnitudes should be higher than this 
            for j in range(-3, 4):
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                for item in dlist:
                    if mag_dict[item] < min_mag:
                        min_mag = mag_dict[item] # update brightest available star
                        cind = j + 3 # column index
                        sname = item # star name 
            if len(sname) > 1: # if it's found *no* stars, leave blank                
                df_magname.at[i, cind] = sname        
        # CASE 2: one known star,  choose that one and add to dictionary
        elif len(scand_list) == 1:
            sname = scand_lit[0]
            for j in range(-3, 4): # iterate through columns in row ( = horizon bins)
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                if sname in dlist:
                    cind = j + 3 # column index
                    df_magname.at[i, cind] = sname 
            known_stars[sname] = "K" + str(len(known_stars)).zfill(2) # update known star dictionary 
        # CASE 3: several known stars, choose the brightest one 
        else:
            # first find the brightest available star
            min_mag = 10 # all human visible magnitudes should be higher than this 
            for scand in scand_list:
                if mag_dict[scand] < min_mag:
                    min_mag = mag_dict[scand] # update brightest available star
                    sname = item # star name 
            known_stars[sname] = "K" + str(len(known_stars)).zfill(2) # update known star dictionary 
            for j in range(-3, 4): # now find position of star
                dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
                if sname in dlist:
                    cind = j + 3 # column index
                    df_magname.at[i, cind] = sname 
    # return                 
    df_magname.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_magname, known_stars)

In [102]:
(df_test, known_star_test) = name_or_mag_data(df, mag_dict, known_stars)

In [104]:
df_test

,-3,-2,-1,0,1,2,3
0,,,,H003419,,,
1,,,,,,H003419,
2,,,,,,,H008102
3,,,H018543,,,,
4,H025606,,,,,,
5,H030324,,,,,,
6,H030438,,,,,,
7,,,,,,H032349,
8,,,,H030438,,,
9,,,,,,H030438,


In [103]:
known_star_test

{}

In [94]:
known_stars = {}
len(known_stars)

0

In [97]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

str(len(thisdict)).zfill(2)

'03'

In [93]:
df_test = name_or_mag_data(testd_df_save, mag_dict, {})

H008645
H008102
H005364
H003419
H009347
H008102
H003419
H016537
H017378
H008102
H019747
H020042
H018543
H025606
H027628
H030324
H030438
H038414
H039429
H030438
H032349
H041037
H039953
H030438
H045101
H046701
H048002
H045556
H041037
H039953
H030438
H052419
H052727
H045556
H041037
H039953
H030438
H059747
H060718
H065936
H066657
H068702


In [83]:
scand_list = []
len(scand_list)

0

In [87]:
testd_df_save

,-3,-2,-1,0,1,2,3
0,H008645 H009347,H008102,H005364 H006537,H003419,,H001562,
1,,,H009347,H008102 H008645,H006537,H003419 H005364,
2,H016537 H017378,H014879 H015474,H013701,,H009347,H008645,H008102 H009487
3,H019747 H020042 H020535 H021594,,H018543,H014879 H015474 H016537 H017378,,H013701,H009347
4,H025606,H019747 H021393 H023685 H024305,H020042 H020535 H021594,,H018543,H014879 H015474,H016537 H017378
5,H027628 H028328 H030324,H025859 H026634 H027072 H027654 H028103,H025606 H025985 H027288,H019747 H020535 H021393 H023685 H024305,H020042,H021594,H014879
6,H030438 H031685 H032768 H033579 H033856 H03397...,H032759 H033152,H027628 H028328 H030122 H030277 H031592,H025859 H026634 H030324,H019747 H027072 H027654,H020042 H020535 H021393 H025606 H025985 H02728...,H023685 H024305 H027366
7,H038414 H039429 H039757,H032768 H036377 H037677 H037819 H038170,H030438 H031685 H035264 H035904 H037229,H028328 H032759 H033579 H033856 H034444,H030122 H030277 H033152 H033977,H025859 H026634 H027628 H031592 H032349,H019747 H027654 H030324
8,H041037 H042536 H042570 H042913 H043023 H04451...,H038827 H039953 H042515 H042828,H038414 H039429,H030438 H032768 H036377 H037819,H031685 H035264 H037229 H037677 H038170 H039757,H028328 H032759 H035904,H027628 H030122 H030277 H033152 H033579 H03385...
9,H045101 H046701 H047854 H048002 H048774,H043783 H045080 H045556 H045941 H046651,H041037 H042536 H042570 H042913 H043023 H04451...,H038827 H039953,H032768 H039429 H042515 H042828,H030438 H036377 H037819 H038414,H028328 H031685 H035264 H037677 H039757


In [34]:
testd_df_save.shape

(13, 7)

In [54]:
list(filter(None, testd_df_save[-3][0].split(' '))) 

['H008645', 'H009347']

In [45]:
df_test = mag_or_name_data(testd_df_save, mag_dict, {})

In [43]:
df_test

,-3,-2,-1,0,1,2,3
0,,,,H003419,,,
1,,,,,,H003419,
2,,,,,,,H008102
3,,,H018543,,,,
4,H025606,,,,,,
5,H030324,,,,,,
6,H030438,,,,,,
7,,,,,,H032349,
8,,,,H030438,,,
9,,,,,,H030438,


In [48]:
if 'H000678' in mag_dict:
    print("yes")
else:
    print("no")    

no


In [38]:
mag_dict

{'H000677': 2.07,
 'H000746': 2.28,
 'H000765': 3.88,
 'H001067': 2.83,
 'H001562': 3.56,
 'H002021': 2.82,
 'H002072': 3.93,
 'H002081': 2.4,
 'H002920': 3.69,
 'H003092': 3.27,
 'H003179': 2.24,
 'H003419': 2.04,
 'H003821': 3.46,
 'H004427': 2.15,
 'H004436': 3.86,
 'H005165': 3.32,
 'H005348': 3.94,
 'H005364': 3.46,
 'H005447': 2.07,
 'H006537': 3.6,
 'H006686': 2.66,
 'H006867': 3.41,
 'H007083': 3.93,
 'H007097': 3.62,
 'H007588': 0.45,
 'H007607': 3.59,
 'H008102': 3.49,
 'H008645': 3.74,
 'H008796': 3.42,
 'H008832': 3.88,
 'H008886': 3.35,
 'H008903': 2.64,
 'H009007': 3.69,
 'H009236': 2.86,
 'H009347': 3.99,
 'H009487': 3.82,
 'H009598': 3.95,
 'H009640': 2.1,
 'H009884': 2.01,
 'H010064': 3.0,
 'H010602': 3.56,
 'H011767': 1.97,
 'H012706': 3.47,
 'H013209': 3.61,
 'H013268': 3.77,
 'H013531': 3.93,
 'H013701': 3.89,
 'H013847': 2.88,
 'H014135': 2.54,
 'H014328': 2.91,
 'H014354': 3.32,
 'H014576': 2.09,
 'H014668': 3.79,
 'H014879': 3.8,
 'H015474': 3.7,
 'H015863': 1.79